In [2]:
import os
%pwd

'f:\\iNeuron\\End2End\\Bappy E-2-E\\End_to_End_ML_Project\\research'

In [3]:
os.chdir('f:\\iNeuron\\End2End\\Bappy E-2-E\\End_to_End_ML_Project')

In [4]:
%pwd

'f:\\iNeuron\\End2End\\Bappy E-2-E\\End_to_End_ML_Project'

In [25]:
from dataclasses import dataclass #type: ignore
from pathlib import Path #type: ignore

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train: Path
    test: Path
    model: str
    alpha: float
    l1_ratio: float
    target: str

In [26]:
from src.mlproject.constants import *
from src.mlproject.utils.common import load_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_path = CONFIG_PATH,
                 params_path = PARAMS_PATH,
                 schema_path = SCHEMA_PATH) -> None:
        self.config = load_yaml(config_path)
        self.params = load_yaml(params_path)
        self.schema = load_yaml(schema_path)

        create_directories(self.config.model_trainer.root_dir)

    def get_model_trainer_config(self):
        config = self.config.model_trainer
        params = self.params.Elastic_net
        schema = self.schema.TARGET_COLUMN
        model_trainer_obj = ModelTrainerConfig(
                                                root_dir= config.root_dir,
                                                train= config.train_path,
                                                test = config.test_path,
                                                model= config.model_name,
                                                alpha = params.alpha,
                                                l1_ratio= params.l1_ratio,
                                                target = schema.name)
        return (model_trainer_obj)

In [27]:
import pandas as pd
import joblib
from sklearn.linear_model import ElasticNet

class model_trainer:
    def __init__(self, config: ModelTrainerConfig) -> None:
        self.config = config

    def train(self):
        train = pd.read_csv(self.config.train)
        test = pd.read_csv(self.config.test)

        xtrain = train.drop(columns = self.config.target)
        xtest = test.drop(columns = self.config.target)

        ytrain = train[self.config.target]
        ytest = test[self.config.target]

        elastic_net = ElasticNet(alpha= self.config.alpha,
                                 l1_ratio= self.config.l1_ratio,
                                 random_state= 8)
        
        elastic_net.fit(xtrain,ytrain)

        joblib.dump(elastic_net,self.config.model)

In [28]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_obj = model_trainer(config= model_trainer_config)
    model_trainer_obj.train()

except Exception as e:
    raise e

[2023-11-02 20:46:08,851: INFO: common: The YAML file: config\config.yaml is loaded]
[2023-11-02 20:46:08,854: INFO: common: The YAML file: params.yaml is loaded]
[2023-11-02 20:46:08,858: INFO: common: The YAML file: schema.yaml is loaded]
[2023-11-02 20:46:08,859: INFO: common: Directory:artifacts/model_trainer is created]
